## Fetch ground station data

In [1]:
# Insert observation_id's
observation_ids = [169356,
                   169357,
                   169360,
                   169348,
                   169345,
                   169362,
                   169353,
                   169350,
                   169346,
                   169351,
                   169349,
                   169354,
                   169096,
                   169355,
                   168732,
                   169347]

In [2]:
# Get station location from the observation via the observation_id

import requests

NETWORK_BASE_URL = 'https://network.satnogs.org/api'
observations = []
for observation_id in observation_ids:
    r = requests.get(url='{}/observations/{}/'.format(NETWORK_BASE_URL, observation_id))
    if r.status_code != requests.codes.ok:
        print("Observation {} not found in network.".format(observation_id))
    observations.append(r.json())

In [3]:
# Create ground station GeoJSON feature collection
import geojson

gs_features = []
for observation in observations:
    ob_feature = geojson.Feature(geometry=geojson.Point((observation['station_lng'],
                                                         observation['station_lat'])),
                                 properties=observation)
    gs_features.append(ob_feature)
# gs_collection = geojson.FeatureCollection(features)

## Fetch satellite data

In [4]:
# Find the first start time and the last end time
from datetime import datetime, date


start_time_str = sorted(observations, key=lambda obs: obs['start'])[0]['start']
end_time_str = sorted(observations, key=lambda obs: obs['end'])[-1]['end']

start_time = datetime.strptime(start_time_str, '%Y-%m-%dT%H:%M:%SZ')
end_time = datetime.strptime(end_time_str, '%Y-%m-%dT%H:%M:%SZ')

In [5]:
# Get TLE of the satellite
# NOTE: Couldn't find an API endpoint for this,
# leave this code here for the future

norad_cat_id = observations[0]['norad_cat_id']
DB_BASE_URL = 'https://db.satnogs.org/api'

r = requests.get(url='{}/satellites/{}/'.format(DB_BASE_URL, norad_cat_id))
if r.status_code != requests.codes.ok:
    print("ERROR: Satellite {} not found in network.".format(norad_cat_id))
satellite_meta = r.json()
satellite_meta['start'] = start_time_str
satellite_meta['end'] = end_time_str

In [6]:
# Calculate satellite ground track using pyephem
# (Fetching the TLE wasn't possible, use hardcoded TLE instead)
import ephem
from datetime import datetime


# Return 'num' evenly spaced datetimes in the range from 'start' to 'stop'
def lin_datetime_range(start, end, num=50):
    time_step = (end - start)/num
    for n in range(num):
        yield start + n * time_step

line1 = "ISS (ZARYA)"

# Source: https://network.satnogs.org/observations/169360/
line2 = '1 25544U 98067A   18177.59501089  .00001867  00000-0  35640-4 0  9995'
line3 = '2 25544  51.6402 331.8141 0003663 229.9897 168.6155 15.53970531119984'

satellite = ephem.readtle(line1, line2, line3)
sat_positions = []
for t in lin_datetime_range(start_time, end_time, num=50):
    satellite.compute(t)
    sat_positions.append((satellite.sublong/ephem.degree,
                         satellite.sublat/ephem.degree))
sat_linestring = geojson.LineString(sat_positions)
sat_feature = geojson.Feature(geometry=sat_linestring, properties=satellite_meta)

## Combine both Features

In [7]:
# Combine ground station and satellite features
collection = geojson.FeatureCollection([*gs_features, sat_feature])

In [8]:
# Write GeoJSON collection in a file
import json

# data = geojson.dumps(collection, sort_keys=True)
GEOJSON_OUTPUT = 'station_positions.geojson'
with open(GEOJSON_OUTPUT, 'w') as out_file:
    json.dump(collection, out_file)

In [9]:
# Display the GeoJSON file

import folium

# Load the GeoJSON file
with open(GEOJSON_OUTPUT, 'rb') as in_file:
    data = json.load(in_file)

# Create a map centered on the first point
m = folium.Map(location=[47.72, 14.68],
               zoom_start=4)

folium.GeoJson(
    data,
    name='geojson'
).add_to(m)

m